In [1]:
!git clone https://github.com/zer0int/CLIP-fine-tune.git

Cloning into 'CLIP-fine-tune'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 33 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (33/33), 22.52 KiB | 512.00 KiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
%cd CLIP-fine-tune/

/workspace/CLIP-fine-tune


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!pip install -r requirements-finetune.txt

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-n69skk0r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-n69skk0r
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 29.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 21.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.7 MB/s et

In [5]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [8]:
import clip
import torch

clipmodel = 'ViT-L/14'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clipmodel, device=device)

100%|███████████████████████████████████████| 890M/890M [00:19<00:00, 48.9MiB/s]


In [9]:
!wget https://huggingface.co/toady123/coord-tracker/resolve/main/clip_ft_30.pt && mv clip_ft_30.pt models/.

--2024-05-05 09:19:13--  https://huggingface.co/toady123/coord-tracker/resolve/main/clip_ft_30.pt
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.16, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/57/7f/577f0c751666b7dbd1175d2d06522b943536bd62b45b60160b10d6f0b573a558/a9aa59dc7ffe957a15fd8649ec058152b959fefee736900940214bec04987940?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27clip_ft_30.pt%3B+filename%3D%22clip_ft_30.pt%22%3B&Expires=1715159953&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTE1OTk1M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzU3LzdmLzU3N2YwYzc1MTY2NmI3ZGJkMTE3NWQyZDA2NTIyYjk0MzUzNmJkNjJiNDViNjAxNjBiMTBkNmYwYjU3M2E1NTgvYTlhYTU5ZGM3ZmZlOTU3YTE1ZmQ4NjQ5ZWMwNTgxNTJiOTU5ZmVmZWU3MzY5MDA5NDAyMTRiZWMwNDk4Nzk0MD9y

In [11]:
# Load the fine-tuned model
model_path = 'models/clip_ft_40.pt'  # Update this path
model = torch.load(model_path)
model.eval()
model_path

'models/clip_ft_30.pt'

In [12]:
import os

def gather_prompts(directory):
    prompts = set()  # Using a set to avoid duplicate prompts

    # Walk through all files in the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):  # Ensure it's a text file
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    # Read each line from the file, strip whitespace, and add to the set
                    for line in f:
                        stripped_line = line.strip()
                        #print(stripped_line)
                        if stripped_line:  # Only add non-empty lines
                            prompts.add(stripped_line)

    return list(prompts)  # Convert set to list to allow indexing

# Specify the path to your directory containing text files
directory_path = 'tiles/'
all_prompts = gather_prompts(directory_path)

print(f"Found {len(all_prompts)} unique prompts.")


Found 255 unique prompts.


In [13]:
# Assuming 'model', 'preprocess', and 'image_preprocessed' are already defined
text_tokens = clip.tokenize(all_prompts).to(device)

# Proceed with model encoding and similarity calculation
with torch.no_grad():
    #image_features = model.encode_image(image_preprocessed)
    text_features = model.encode_text(text_tokens)



In [20]:
from PIL import Image
# Load the image
image_path = 'tiles/1536_2048.png'  # Adjust this path
image = Image.open(image_path).convert("RGB")

# Preprocess the image
image_preprocessed = preprocess(image).unsqueeze(0).to(device)

# Proceed with model encoding and similarity calculation
with torch.no_grad():
    # Calculate the cosine similarity
    logits_per_image, logits_per_text = model(image_preprocessed, text_tokens)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# Display the top scoring prompts
top_prompts = sorted(zip(all_prompts, probs[0]), key=lambda x: x[1], reverse=True)[:5]  # Top 5 predictions
for prompt, probability in top_prompts:
    print(f"{prompt}: {probability:.4f}")


(-97.74734405076143, 30.28421523530908),mysterious group of flamingos urban called a synthetic: 0.4884
(-97.72553683587141, 30.281803065377765),You can hear a urban whale's heartbeat from more sacred 2 miles ancient: 0.2472
(-97.7099602538071, 30.29386391503434),A solar of flamingos is mythic a flamboyance.: 0.0293
(-97.74422873434857, 30.278184810480795),solar lived solar in time to the moon landing than exotic the barren of the Great legendary crimson Giza.: 0.0227
(-97.75357468358715, 30.278184810480795),fabricated group luminous flamingos wonderful urban a flamboyance.: 0.0184
